In [36]:
import requests #імпортуємо бібліотеку, яка дозволяє нам робити запити та працювати із АРІ
from pprint import pprint as pp # pprint - бібліотека, яка дозволяє зручно переглядати ієрархічні дані
import sqlite3 # бібліотека для створення та роботи із sqlite базами даних
import pandas as pd #бібліотека для роботи із масивами та таблицями
from tqdm import tqdm # бібліотека для створення прогресбарів
from datetime import date #бібліотека для роботи із часом
# import seaborn as sns #бібліотека для візуалізації даних
import os

from pymongo import MongoClient
from IPython.display import clear_output 


### Куди копати

* Аналіз допорогових закупівель
* Активний аукціон система пропонує постачальників, система пінгує
* Ковідні закупівлі - окрема тема

In [37]:
from urllib.parse import urlparse, parse_qsl, urljoin
import datetime

tz = datetime.timezone(datetime.timedelta(seconds=10800))
start_date = datetime.datetime(2021,1,1,tzinfo=tz)
end_date = datetime.datetime(2021,10,1,tzinfo=tz)

api_path = "https://public.api.openprocurement.org/api/2.5/"

next_page_url = urljoin(api_path,f"tenders?offset={start_date.isoformat()}")
offset_date = start_date

tender_ids = []

while offset_date<end_date:

    clear_output()
    print(f'Date {offset_date} tender ids {len(tender_ids) }')

    page = requests.get(next_page_url).json()
    
    # new next page
    next_page_url = page["next_page"]["uri"]    

    tender_ids += [tender["id"] for tender in page["data"]]

    qs = dict(
        parse_qsl(
            urlparse(next_page_url).query
            )
        )

    # to compare with end date
    offset_date = datetime.datetime.fromisoformat(qs['offset'])

# unique
tender_ids = list(set(tender_ids))

Date 2021-03-17 13:53:34.857358+02:00 tender ids 1002100


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
with open('./scraper/tender_ids.txt', 'w') as f:
    f.writelines(s + '\n' for s in tender_ids)

In [163]:
with open('./scraper/tender_ids.txt', 'r') as f:
    tender_ids = [line.strip() for line in f.readlines()]

len(tender_ids)

476400

In [19]:
client = MongoClient()
db = client['prozorro']
tenders_collection = db['tenders']

## USE SPIDER - work faster and has outothrottle mechanism
# def get_tender(tender_id):
#     response = requests.get(
#         urljoin(api_path+'tenders/', tender_id)
#     ).json()

#     tender = response["data"]

#     del tender['id']
    
#     update_info = tenders_collection.update({'_id':tender_id}, tender, upsert=True)
#     return update_info

# # slow
# for tender_id in tqdm(tender_ids):
    
#     get_tender(tender_id)


In [172]:
import concurrent.futures

# faster

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     for tender_id, prime in zip(tender_ids, executor.map(get_tender, tender_ids)):
#         clear_output()
#         print('Tender %s is acknowledged: %s' % (tender_id, prime))

In [34]:
# https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/#return-specific-fields-in-embedded-documents

def read_mongo(db, collection, query={}, fields=None, id_to_index=False):
    """ Read from Mongo and Store into DataFrame """

    # Make a query to the specific DB and Collection
    
    # filter fields
    _fields = None
    if fields is not None:
        _fields = fields if isinstance(fields, dict) else {field:1 for field in fields}

    cursor = db[collection].find(query, _fields)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if id_to_index:
        df.set_index('_id')

    return df

# You can put filter here or later in pandas dataset
# equvalent to
# SELECT fields
#   FROM collection
#   WHERE query
df = read_mongo(db, collection='tenders', query={'status':'complete'}, fields=['status', 'items'])


In [35]:
df.head()

,_id,items,status
0,94578d695cf6482a9578ebfd4582a8a2,"[{'id': '8b7380d2e4154936b4f3dbfaa1b20964', 'd...",complete
1,b39bfd4450d840e69d17df7c41a56b81,"[{'id': 'dd18091a31f9483996743cf874a5d212', 'd...",complete
2,c71c182bf9e54cd79b980a5380f58a38,"[{'id': '1fbf147e10f246d9b7c07521e6d1fb12', 'd...",complete
3,85e7d85d922d4e3a9f2719c2da54c55b,"[{'id': '9880cbb7cd7d487190eb9e00c1e22e84', 'd...",complete
4,1240a20d6c7048819d9b9cb1b982075c,"[{'id': '07c81b6a77baf5cd17476028ea22c5f5', 'd...",complete


**tender Schema**

[here](https://gist.github.com/imaginal/bf19e659b34b6acd7ee002f6b88dbf81)

interesting example

https://public.api.openprocurement.org/api/2.5/tenders/98d54c150056474fbedb5420f8aad0d9

In [16]:
df.head()

,_id,title,documents,description,date,dateModified,tenderID,owner,mainProcurementCategory,plans,...,buyers,title_en,qualifications,qualificationPeriod,complaints,profile,shortlistedFirms,noticePublicationDate,features,cancellations
0,94578d695cf6482a9578ebfd4582a8a2,Продукти харчування.,"[{'id': '62236625922b40dd9fb5fd9c02ad7967', 'h...",Кожна партія товару повинна мати маркування з...,2021-09-01T11:05:02.555508+03:00,2021-09-01T11:05:02.555508+03:00,UA-2021-09-01-001589-a,prom.ua,goods,[{'id': 'a3af1e5196c54833919d968779a84be4'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b39bfd4450d840e69d17df7c41a56b81,Послуги з юридичного консультування та юридичн...,"[{'id': '7c557f571e1f45c8bd02e2bb1413939f', 'h...",надавати безоплатну вторинну правову допомогу ...,2021-09-01T09:03:34.877174+03:00,2021-09-01T09:03:34.877174+03:00,UA-2021-09-01-000303-a,prom.ua,services,[{'id': '3490a5c0f5044184a7bf8bec7ae2ac84'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c71c182bf9e54cd79b980a5380f58a38,Холодильник GRUNHELM GRW-143DD,"[{'id': '9934723c90c749d29d4750423646b2ef', 'h...",NaN,2021-09-01T11:42:37.074993+03:00,2021-09-01T11:42:37.074993+03:00,UA-2021-09-01-002597-a,it.ua,goods,[{'id': '2ca552bc87bb40828992ec25f276e5c8'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,85e7d85d922d4e3a9f2719c2da54c55b,Шлак,"[{'id': '56cfadbf332843a080b9e63bff05b8a7', 'h...",відсутня,2021-09-01T10:11:38.632843+03:00,2021-09-01T10:11:38.632843+03:00,UA-2021-09-01-000916-a,prom.ua,goods,[{'id': '16e93734a4084fbe9b50ea73b821edc1'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1240a20d6c7048819d9b9cb1b982075c,код ДК 021:2015: 09120000-6 Газове паливо (При...,"[{'id': '21b6eaceb6764e51877857d59fec243e', 'h...",код ДК 021:2015: 09120000-6 Газове паливо (При...,2021-09-01T09:07:25.531109+03:00,2021-09-01T09:07:25.531109+03:00,UA-2021-08-26-017859-a,netcast.com.ua,goods,[{'id': '1bd2af50e9f14903b1b528c0a0d62876'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
len(df.columns)

48